ttempting to delete all hardcoding from beambeam3d.py and having variables as inputs instead

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import xobjects as xo
import xtrack as xt
import xfieldsdevlumi as xf
import xpart as xp
import time
# Generating sequences

In [2]:
context = xo.ContextCpu()

In [3]:
p0c = 6500e9
bunch_intensity = 0.7825E11
physemit_x = (2.946E-6*xp.PROTON_MASS_EV)/p0c 
physemit_y = (2.946E-6*xp.PROTON_MASS_EV)/p0c 
beta_x = 19.17
beta_y = 19.17
sigma_z = 0.08
sigma_delta = 1E-4
beta_s = sigma_z/sigma_delta
Qx = 64.31
Qy = 59.32
Qs = 2.1E-3
frev = 11245.5 
nTurn = 3

In [4]:
n_macroparticles = int(1e4)
xs_b1 = []
ys_b1 = []
xs_b2 = []
ys_b2 = []
pxs_b1 = []
pys_b1 = []
pxs_b2 = []
pys_b2 = []

In [5]:
pipeline_manager = xt.PipelineManager()
pipeline_manager.add_particles('b1',0)
pipeline_manager.add_particles('b2',0)
pipeline_manager.add_element('IP1')
pipeline_manager.add_element('IP2')

In [6]:
particles_b1 = xp.Particles(_context=context,
    p0c=p0c,
    x=np.sqrt(physemit_x*beta_x)*(np.random.randn(n_macroparticles)),
    px=np.sqrt(physemit_x/beta_x)*np.random.randn(n_macroparticles),
    y=np.sqrt(physemit_y*beta_y)*(np.random.randn(n_macroparticles)),
    py=np.sqrt(physemit_y/beta_y)*np.random.randn(n_macroparticles),
    zeta=sigma_z*np.random.randn(n_macroparticles),
    delta=sigma_delta*np.random.randn(n_macroparticles),
    weight=bunch_intensity/n_macroparticles
)
particles_b1.init_pipeline('b1')
particles_b2 = xp.Particles(_context=context,
    p0c=p0c,
    x=np.sqrt(physemit_x*beta_x)*(np.random.randn(n_macroparticles)),
    px=np.sqrt(physemit_x/beta_x)*np.random.randn(n_macroparticles),
    y=np.sqrt(physemit_y*beta_y)*(np.random.randn(n_macroparticles)),
    py=np.sqrt(physemit_y/beta_y)*np.random.randn(n_macroparticles),
    zeta=sigma_z*np.random.randn(n_macroparticles),
    delta=sigma_delta*np.random.randn(n_macroparticles),
    weight=bunch_intensity/n_macroparticles
)
particles_b2.init_pipeline('b2')

###########<br>
Beam-beam #<br>
###########

In [7]:
slicer = xf.TempSlicer(sigma_z=sigma_z, n_slices=1, mode = 'shatilov')
config_for_update_b1 = xf.ConfigForUpdateBeamBeamBiGaussian3D(
pipeline_manager=pipeline_manager,
element_name='IP1',
partner_particles_name = 'b2',
slicer=slicer,
update_every=1,
)
config_for_update_b2 = xf.ConfigForUpdateBeamBeamBiGaussian3D(
pipeline_manager=pipeline_manager,
element_name='IP1',
partner_particles_name = 'b1',
slicer=slicer,
update_every=1,
)

Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.


In [8]:
IP2_config_for_update_b1 = xf.ConfigForUpdateBeamBeamBiGaussian3D(
pipeline_manager=pipeline_manager,
element_name='IP2',
partner_particles_name = 'b2',
slicer=slicer,
update_every=1,
)
IP2_config_for_update_b2 = xf.ConfigForUpdateBeamBeamBiGaussian3D(
pipeline_manager=pipeline_manager,
element_name='IP2',
partner_particles_name = 'b1',
slicer=slicer,
update_every=1,
)

In [9]:
print('build bb elements...')
bbeam_b1 = xf.BeamBeamBiGaussian3D(
            _context=context,
            other_beam_q0 = particles_b2.q0,
            phi = 0,alpha=0,
            config_for_update = config_for_update_b1,
            flag_luminosity = 1,
            flag_numerical_luminosity = 1,
            n_lumigrid_cells=1500,
            sig_lumigrid_cells=np.sqrt(physemit_x*beta_x),
            range_lumigrid_cells = 24,
            n_macroparticles=n_macroparticles,
            nTurn = nTurn,
            update_lumigrid_sum=1
            )
bbeam_b2 = xf.BeamBeamBiGaussian3D(
            _context=context,
            other_beam_q0 = particles_b1.q0,
            phi = 0,alpha=0,
            config_for_update = config_for_update_b2,
            flag_luminosity = 1,
            flag_numerical_luminosity = 1,
            n_lumigrid_cells=1500,
            sig_lumigrid_cells=np.sqrt(physemit_x*beta_x),
            range_lumigrid_cells = 24,
            n_macroparticles=n_macroparticles,
            nTurn = nTurn,
            update_lumigrid_sum=1
            )
print(n_macroparticles)
IP2_bbeam_b1 = xf.BeamBeamBiGaussian3D(
            _context=context,
            other_beam_q0 = particles_b2.q0,
            phi = 0,alpha=0,
            config_for_update = IP2_config_for_update_b1,
            flag_luminosity = 1,
            flag_numerical_luminosity = 1,
            update_lumigrid_sum=1)
IP2_bbeam_b2 = xf.BeamBeamBiGaussian3D(
            _context=context,
            other_beam_q0 = particles_b1.q0,
            phi = 0,alpha=0,
            config_for_update = IP2_config_for_update_b2,
            flag_luminosity = 1,
            flag_numerical_luminosity=1,
            update_lumigrid_sum=1)

build bb elements...
10000


###############################################################<br>
arcs (here they are all the same with half the phase advance) #<br>
###############################################################

In [10]:
arc = xt.LineSegmentMap(
        betx = beta_x,bety = beta_y,
        qx = Qx/2, qy = Qy/2,bets = beta_s, qs=Qs)
#################################################################
# Tracker                                                       #
#################################################################
'''
elements_b1 = [bbeam_b1,arc]
elements_b2 = [bbeam_b2,arc]
element_names_b1 = ['bbeam_b1','arc']
element_names_b2 = ['bbeam_b2','arc']
'''
elements_b1 = [bbeam_b1,arc, IP2_bbeam_b1]
elements_b2 = [bbeam_b2,arc, IP2_bbeam_b2]

In [11]:
line_b1 = xt.Line(elements=elements_b1)
line_b2 = xt.Line(elements=elements_b2)
line_b1.build_tracker()
line_b2.build_tracker()
branch_b1 = xt.PipelineBranch(line_b1,particles_b1)
branch_b2 = xt.PipelineBranch(line_b2,particles_b2)

In [12]:
multitracker = xt.PipelineMultiTracker(branches=[branch_b1,branch_b2])

In [13]:
record_qss_b1 = line_b1.start_internal_logging_for_elements_of_type(xf.BeamBeamBiGaussian3D, 
                                                        capacity={
                                                            "beamstrahlungtable": int(0),
                                                            "bhabhatable": int(0),
                                                            "lumitable": nTurn,
                                                            "numlumitable": nTurn
                                                        })

In [14]:
print('Tracking...')
time0 = time.time()

Tracking...


In [15]:
multitracker.track(num_turns=nTurn,turn_by_turn_monitor=True)
print('Done with tracking.',(time.time()-time0)/10,'[s/turn]')
line_b1.stop_internal_logging_for_elements_of_type(xf.BeamBeamBiGaussian3D)

Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.
lumigrid_my_beam size: (1500, 1500)
exchange_buffer size: (2250017,)
lumigrid_my_beam size: (1500, 1500)
exchange_buffer size: (2250017,)


IndexError: index 0 is out of bounds for axis 0 with size 0

In [16]:
record_qss_b1.move(_context=xo.context_default)

In [17]:
lumi_b1_beambeam = record_qss_b1.lumitable.luminosity
num_lumi_b1_beambeam = bbeam_b1.numlumitable.numerical_luminosity
grid = bbeam_b1.lumigrid_sum
print('Gaussian Luminosity with beam-beam:',frev*lumi_b1_beambeam)
print('Numerical Integrator Luminosity with beam-beam:',num_lumi_b1_beambeam)
print(np.sum(grid))
def Lumi_analytical(Nb, N1, N2, frev, Delta_i, sig_i, sig_x, sig_y):
    W = np.exp(-Delta_i**2/(4*sig_i**2))
    return ((Nb * N1 * N2 * frev * W)/(4 * np.pi * sig_x * sig_y))

Gaussian Luminosity with beam-beam: [0. 0. 0.]
Numerical Integrator Luminosity with beam-beam: [0. 0. 0.]
5025250598172988.0


In [18]:
print('analytical:',Lumi_analytical(1, bunch_intensity, bunch_intensity, frev, 0,np.sqrt(physemit_x*beta_x), np.sqrt(physemit_x*beta_x), np.sqrt(physemit_y*beta_x)))

analytical: 6.72151700827703e+32
